First, with code from http://localhost:8888/tree/Documents/GitHub/aas_workshop_2020_winter

In [2]:
import numpy as np
import astropy.units as u
## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")

## For simple astropy tables
import astropy, io

## For handling ordinary astropy Tables
from astropy.table import Table

## For handling VO table type objects
from astropy.io import votable as apvot

## Generic VO access routines
import pyvo as vo

In [3]:
heasarc_tap_services=vo.regsearch(servicetype='tap',keywords=['heasarc'])
heasarc_tap_services

heasarc_tables=heasarc_tap_services[0].service.tables
for tablename in heasarc_tables.keys():
    if "zcat" in tablename:  
        heasarc_tables[tablename].describe()
        print("Columns={}".format(sorted([k.name for k in heasarc_tables[tablename].columns ])))
        print("----")

abellzcat
    Abell Clusters Measured Redshifts Catalog

Columns=['"__row"', '"__x_ra_dec"', '"__y_ra_dec"', '"__z_ra_dec"', 'abell_distance_class', 'abell_radius', 'abell_richness_class', 'bautz_morgan_class', 'bautz_morgan_flag', 'bii', 'class', 'dec', 'lii', 'log_redshift_ratio', 'mag_10', 'name', 'ra', 'redshift', 'redshift_flag', 'ref_redshift']
----
romabzcat
    Roma-BZCAT Multi-Frequency Catalog of Blazars

Columns=['"__row"', '"__x_ra_dec"', '"__y_ra_dec"', '"__z_ra_dec"', 'bii', 'dec', 'flux_143_ghz', 'flux_1p4_ghz', 'gr_photon_flux', 'lii', 'name', 'object_type', 'ra', 'redshift', 'redshift_flag', 'rmag', 'ro_spectral_index', 'source_number', 'xray_flux']
----
zcat
    CfA Redshift Catalog (June 1995 Version)

Columns=['"__row"', '"__x_ra_dec"', '"__y_ra_dec"', '"__z_ra_dec"', '"distance"', 'bar_type', 'bii', 'bmag', 'bt_mag', 'class', 'comments', 'dec', 'diameter_1', 'diameter_2', 'lii', 'luminosity_class', 'morph_type', 'name', 'notes', 'ra', 'radial_velocity', 'radial_vel

Now with pyvo.registry.regtap, and its connecting *ivoid2service* function

In [4]:
regtap_search_results = vo.registry.regtap.search(keywords=['redshift'])

In [5]:
regtap_search_results[0].ivoid

'ivo://aip.gavo.org/tap'

In [9]:
related_service = vo.registry.regtap.ivoid2service(regtap_search_results[0].ivoid, servicetype="tap")
related_service

In [11]:
related_tables = related_service.tables
for tablename in related_tables.keys():
    related_tables[tablename].describe()
    print("Columns={}".format(sorted([k.name for k in related_tables[tablename].columns ])))
    print("----")

bolshoi.dens256
    Densities for 256^3 grid cells; for some snapnums

Columns=['dens', 'ix', 'iy', 'iz', 'phkey', 'snapnum', 'webid']
----
bolshoi.fof
    Friends-of-Friends halo catalogue, level 0 with relative linking
    length 0.17

Columns=['angmom', 'angmom_x', 'angmom_y', 'angmom_z', 'axis1', 'axis1_x', 'axis1_y', 'axis1_z', 'axis2', 'axis2_x', 'axis2_y', 'axis2_z', 'axis3', 'axis3_x', 'axis3_y', 'axis3_z', 'delta', 'disp', 'disp_v', 'fofid', 'ix', 'iy', 'iz', 'level', 'mass', 'ninfile', 'np', 'phkey', 'size', 'snapnum', 'spin', 'vx', 'vy', 'vz', 'x', 'y', 'z']
----
bolshoi.redshifts
    Redshifts for snapshot numbers

Columns=['aexp', 'snapnum', 'zred']
----
glots.services
    A table of TAP services harvested from the registry (and some
    spoon-fed).

Columns=['accessurl', 'harvestinterval', 'ivoid', 'lastsuccess', 'nextharvest']
----
glots.tables
    A table of tables accesible through the TAP services known to
    glots.services.

Columns=['ivoid', 'table_desc', 'table_na

But why? If we want to organize our resources to normalize table/catalog metadata out from the same record as a service that serves MANY of them (and keep the service record itself shorter, by way of adding one level of indirection).

We know VizieR has resources set up this way.
Let's see what ivoid2service does: https://pyvo.readthedocs.io/en/latest/_modules/pyvo/registry/regtap.html#ivoid2service

In [ ]:
#regtap_search_results = vo.registry.regtap.search(keywords=['vizier'], servicetype='tap')
regtap_search_results = vo.registry.regtap.search(keywords=['vizier'])
for resource in regtap_search_results:
    if 'ivo://cds.vizier' in resource.ivoid:
        print(resource.ivoid)

In [51]:
related_services = vo.registry.regtap.ivoid2service('ivo://CDS.VizieR/J/A+AS/106/373')
related_services

[]

But why isn't this showing? 
http://vao.stsci.edu/directory/getRecord.aspx?id=ivo://CDS.VizieR/J/A+AS/106/373
notes that this is a catalog with a TAP "auxiliary" capability. ivoid2service isn't finding it?

<blockquote>
<type>Catalog</type>
<contentLevel>Research</contentLevel>
<relationship>
<relationshipType>IsServedBy</relationshipType>
<relatedResource ivo-id="ivo://CDS.VizieR/TAP">TAP VizieR generic service</relatedResource>
</relationship>
</content>
<rights>public</rights>
<capability>
<validationLevel validatedBy="ivo://archive.stsci.edu/nvoregistry">2</validationLevel>
<interface xsi:type="vr:WebBrowser">
<accessURL use="full">http://vizier.u-strasbg.fr/cgi-bin/VizieR-2?-source=J/A+AS/106/373</accessURL>
</interface>
</capability>
<capability>
<validationLevel validatedBy="ivo://archive.stsci.edu/nvoregistry">2</validationLevel>
<interface xsi:type="vs:ParamHTTP">
<accessURL use="base">http://vizier.u-strasbg.fr/viz-bin/votable/-dtd/-A?-source=J/A+AS/106/373</accessURL>
<queryType>GET</queryType>
<resultType>text/xml+votable</resultType>
</interface>
</capability>
<capability standardID="ivo://ivoa.net/std/TAP#aux">
<validationLevel validatedBy="ivo://archive.stsci.edu/nvoregistry">2</validationLevel>
<interface role="std" xsi:type="vs:ParamHTTP">
<accessURL use="base">http://tapvizier.u-strasbg.fr/TAPVizieR/tap</accessURL>
</interface>
</capability>
</blockquote>